# 🚀 Indic Language Unlearning - Milestone 1

This notebook tests selective forgetting in multilingual models using:
- **High-resource**: Hindi (hi), Bengali (bn)
- **Low-resource**: Maithili (mai), Bhojpuri (bho)

Using a **lightweight model** BLOOM for faster experimentation.

In [1]:
!pip install -q torch lightning transformers datasets pandas torchmetrics sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 52.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Create project directories
!mkdir -p /content/drive/MyDrive/indic_unlearning/checkpoints_fixed
!mkdir -p /content/drive/MyDrive/indic_unlearning/data
!mkdir -p /content/drive/MyDrive/indic_unlearning/cache

print("✓ Directories created")

✓ Directories created


FLORES - BLOOM

In [19]:
import os
import random
import copy
import torch
import torch.nn.functional as F
import lightning as L
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup
from torchmetrics import Accuracy
import pandas as pd
from google.colab import drive

# 1. Mount Drive (already mounted, but good practice to check)
try:
    drive.mount('/content/drive', force_remount=False)
except:
    print("Drive already mounted.")

# 2. Install required libraries
print("Installing required libraries...")
!pip install -qqq torch lightning transformers datasets sentencepiece pandas torchmetrics

# 3. Create necessary local directory for checkpoints
!mkdir -p .checkpoints/simplified

# --- Configuration (BLOOM Model) ---
DRIVE_DATA_DIR = "/content/drive/MyDrive/indic_unlearning/flores_standard_data"
MODEL_NAME = "bigscience/bloom-560m"  # Changed to BLOOM
TASK = "flores"
FORGET_LANGS = ["te", "bn", "hi"]
MAX_SEQ_LEN = 256
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
EPOCHS = 1
FORGET_NUM = 32
RETAIN_MULTIPLIER = 2
GRADIENT_ACCUMULATION_STEPS = 4

# Dummy hparams object to hold configuration
class SimpleHparams:
    def __init__(self):
        self.model_name_or_path = MODEL_NAME
        self.cache_dir = "./.cache"
        self.task = TASK
        self.forget_lang = FORGET_LANGS
        self.retain_lang = FORGET_LANGS
        self.max_seq_len = MAX_SEQ_LEN
        self.forget_num = FORGET_NUM
        self.retain_multiplier = RETAIN_MULTIPLIER
        self.per_device_train_batch_size = BATCH_SIZE
        self.per_device_eval_batch_size = BATCH_SIZE
        self.num_workers = 0
        self.model_type = "bloom"  # Changed to bloom
        self.output_dir = ".checkpoints/simplified"
        self.learning_rate = LEARNING_RATE
        self.warmup_ratio = 0.1
        self.epochs = EPOCHS
        self.data_dir = DRIVE_DATA_DIR
        self.offline = False
        self.use_flash_attention = False
        self.lr_scheduler_type = "linear"
        self.gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS

    def to_dict(self):
        return {k: v for k, v in self.__dict__.items() if not k.startswith('__')}


def load_json_dataset(args, file_path):
    data_dir = args.data_dir if hasattr(args, 'data_dir') else args.get('data_dir')
    cache_dir = args.cache_dir if hasattr(args, 'cache_dir') else args.get('cache_dir')

    full_path = os.path.join(data_dir, file_path)
    print(f"Loading dataset from: {full_path}")
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"Data file not found at: {full_path}. Check Drive path/permissions.")

    return load_dataset(
        "json",
        data_files=full_path,
        cache_dir=cache_dir,
    )["train"]

# --- FLORESDataset (Sequence-to-sequence style) ---
class FLORESDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_len=256, lang=["en"]):
        self.data = data
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.lang = lang if isinstance(lang, list) else [lang]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        lang = random.choice(self.lang)
        data_row = self.data[idx]
        item = data_row[lang]

        inputs = self.tokenizer(
            item,
            max_length=self.max_seq_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        labels = inputs["input_ids"].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels.squeeze(),
            "lang": lang,
        }


# --- FLORESDataModule ---
class FLORESDataModule(L.LightningDataModule):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir, local_files_only=args.offline)
        self.flores_test = []

    def setup(self, stage=None):
        self.flores_test = []

        forget_data = load_json_dataset(self.args, f"forget-{self.args.forget_num}.jsonl")
        retain_data = load_json_dataset(self.args, f"retain-{self.args.forget_num}-x{self.args.retain_multiplier}.jsonl")

        # FIXED: Train on forget data (not retain data)
        self.flores_forget = FLORESDataset(forget_data, self.tokenizer, self.args.max_seq_len, lang=self.args.forget_lang)

        test_data = load_json_dataset(self.args, "test.jsonl")

        for lang in self.args.forget_lang:
            self.flores_test.append(FLORESDataset(test_data, self.tokenizer, self.args.max_seq_len, lang))
            self.flores_test.append(FLORESDataset(forget_data, self.tokenizer, self.args.max_seq_len, lang))

    def train_dataloader(self):
        # FIXED: Return forget dataloader
        return DataLoader(self.flores_forget, batch_size=self.args.per_device_train_batch_size, shuffle=True, num_workers=self.args.num_workers)

    def test_dataloader(self):
        return [DataLoader(dataset, batch_size=self.args.per_device_eval_batch_size, shuffle=False, num_workers=self.args.num_workers)
                for dataset in self.flores_test]

# --- MultilingualModel with MA and CSV Hook ---
class MultilingualModel(L.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        self.tokenizer = AutoTokenizer.from_pretrained(hparams['model_name_or_path'], cache_dir=hparams['cache_dir'])
        self.model = AutoModelForCausalLM.from_pretrained(hparams['model_name_or_path'], cache_dir=hparams['cache_dir'])

        # Freeze embedding layers for stability against aggressive unlearning
        if hasattr(self.model.base_model, 'embed_tokens'):
            for param in self.model.base_model.embed_tokens.parameters():
                param.requires_grad = False
        if hasattr(self.model.base_model, 'embed_positions'): # Common in XGLM/BART-like
            for param in self.model.base_model.embed_positions.parameters():
                param.requires_grad = False

        print("\n*** Model Layer Freezing Status ***")
        for name, param in self.model.named_parameters():
            if 'embed' in name:
                print(f"  {name}: requires_grad={param.requires_grad}")
        print("***********************************\n")

        self.accuracy = Accuracy(task="multiclass", num_classes=self.tokenizer.vocab_size, ignore_index=-100)

        self.test_dataset_names = []
        for lang in hparams['forget_lang']:
            self.test_dataset_names.append(f"test/{lang}_test_")
            self.test_dataset_names.append(f"test/{lang}_forget_")

    def forward(self, **inputs):
        return self.model(input_ids=inputs["input_ids"], attention_mask=inputs.get("attention_mask"), labels=inputs.get("labels"))

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        # FIXED: Unlearning step - maximize loss on FORGET data (negative gradient)
        loss = loss * -1
        self.log("train/forget_loss", loss, on_step=True, prog_bar=True)

        return loss

    def _calculate_ma(self, batch):
        labels = batch["labels"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]

        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        logits = outputs.logits[..., :-1, :]
        target_labels = labels[..., 1:]

        preds = torch.argmax(logits, dim=-1)

        target_labels_flat = target_labels.flatten()
        preds_flat = preds.flatten()

        valid_indices = (target_labels_flat != -100).nonzero(as_tuple=True)[0]
        preds_valid = preds_flat[valid_indices]
        labels_valid = target_labels_flat[valid_indices]

        if labels_valid.numel() == 0:
            return 0.0

        self.accuracy.to(self.device)
        self.accuracy.update(preds_valid, labels_valid)
        ma = self.accuracy.compute().item()
        self.accuracy.reset()

        return ma

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        loss = outputs.loss
        dataset_name = self.test_dataset_names[dataloader_idx]

        # Clamp loss to a large but finite value to prevent PPL from becoming 'inf'
        MAX_LOSS_FOR_PPL = 11.5129  # log(100000)
        clamped_loss = torch.clamp(loss, max=MAX_LOSS_FOR_PPL)

        ppl = torch.exp(clamped_loss)
        ma = self._calculate_ma(batch)

        _dict = {
            f"{dataset_name}ppl": ppl,
            f"{dataset_name}ma": ma,
        }

        self.log_dict(_dict, on_epoch=True, logger=True)
        return loss

    def on_test_end(self):
        print("\\n--- Saving Test Metrics to CSV ---")
        logged_metrics = self.trainer.logged_metrics

        ppl_metrics = {k: v.item() for k, v in logged_metrics.items() if "ppl" in k}
        ma_metrics = {k: v.item() for k, v in logged_metrics.items() if "ma" in k}

        all_metrics = {**ppl_metrics, **ma_metrics}

        if all_metrics:
            df = pd.DataFrame([all_metrics])
            df.columns = [col.replace("test/", "") for col in df.columns]

            os.makedirs(self.hparams['output_dir'], exist_ok=True)
            output_path = os.path.join(self.hparams['output_dir'], "flores_metrics.csv")
            df.to_csv(output_path, index=False)
            print(f"Metrics successfully saved to {output_path}")
        else:
            print("No test metrics found to save.")

    def configure_optimizers(self):
        # Only optimize parameters that are not frozen (i.e., requires_grad=True)
        trainable_params = filter(lambda p: p.requires_grad, self.model.parameters())
        optimizer = torch.optim.AdamW(trainable_params, lr=self.hparams['learning_rate'])

        # Recalculate training steps based on gradient accumulation
        num_effective_steps_per_epoch = 100 / self.hparams['gradient_accumulation_steps']
        num_training_steps = int(num_effective_steps_per_epoch * self.hparams['epochs'])

        # Ensure num_training_steps is at least 1
        if num_training_steps < 1:
            num_training_steps = 1

        lr_scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(self.hparams['warmup_ratio'] * num_training_steps),
            num_training_steps=num_training_steps,
        )
        return {"optimizer": optimizer, "lr_scheduler": {"scheduler": lr_scheduler, "interval": "step"}}

# --- Main Execution Block ---
if __name__ == "__main__":
    hparams_obj = SimpleHparams()
    hparams = hparams_obj.to_dict()

    print(f"--- Starting FLORES Unlearning Run with BLOOM ---")
    print(f"Task: {hparams['task']}")
    print(f"Model: {hparams['model_name_or_path']}")
    print(f"Target Languages: {hparams['forget_lang']}")
    print(f"Data Directory: {hparams['data_dir']}")
    print(f"Forget/Retain Number: {hparams['forget_num']}")
    print(f"Retain Multiplier: x{hparams['retain_multiplier']}")
    print(f"Learning Rate: {hparams['learning_rate']}")
    print(f"Gradient Accumulation Steps: {hparams['gradient_accumulation_steps']}")
    print(f"\n⚠️  TRAINING ON FORGET SET WITH NEGATIVE LOSS (Unlearning)")

    dm = FLORESDataModule(hparams_obj)
    dm.setup(stage="fit")
    model = MultilingualModel(hparams)

    trainer = L.Trainer(
        default_root_dir=hparams['output_dir'],
        accelerator="auto",
        devices=1,
        max_epochs=hparams['epochs'],
        log_every_n_steps=1,
        enable_checkpointing=False,
        logger=True,
        # Gradient clipping for stability
        gradient_clip_val=0.5,
        # Gradient accumulation for stable batches
        accumulate_grad_batches=hparams['gradient_accumulation_steps'],
    )

    print("\\n--- Starting Training (Unlearning on Forget Set) ---")
    trainer.fit(model, dm)

    print("\\n--- Starting Testing for PPL and MA ---")
    trainer.test(model, dm)

    print("\\n--- Run Complete ---")
    print("Metrics saved to: .checkpoints/simplified/flores_metrics.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Installing required libraries...
--- Starting FLORES Unlearning Run with BLOOM ---
Task: flores
Model: bigscience/bloom-560m
Target Languages: ['te', 'bn', 'hi']
Data Directory: /content/drive/MyDrive/indic_unlearning/flores_standard_data
Forget/Retain Number: 32
Retain Multiplier: x2
Learning Rate: 1e-05
Gradient Accumulation Steps: 4

⚠️  TRAINING ON FORGET SET WITH NEGATIVE LOSS (Unlearning)


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Loading dataset from: /content/drive/MyDrive/indic_unlearning/flores_standard_data/forget-32.jsonl


...

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


\n--- Saving Test Metrics to CSV ---
Metrics successfully saved to .checkpoints/simplified/flores_metrics.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃       DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test/bn_forget_ma     │                           │                           │                           │
│    test/bn_forget_ppl     │                           │                           │                           │
│      test/bn_test_ma      │                           │                           │    0.15384620428085327    │
│     test/bn_test_ppl      │                           │                           │     5029.95263671875      │
│     test/hi_forget_ma     │                           │                           │                           │
│    test/hi_forget_ppl     │                           │                           │                           │
│      test/hi_test_ma      │                           │                           │                           │
│     test/hi_test_ppl      │                           │                           │                           │
│     test/te_forget_ma     │                           │    0.17067307233810425    │                           │
│    test/te_forget_ppl     │                           │     5982.86181640625      │                           │
│      test/te_test_ma      │    0.1623077392578125     │                           │                           │
│     test/te_test_ppl      │      6715.2880859375      │                           │                           │
└───────────────────────────┴───────────────────────────┴───────────────────────────┴───────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 3        ┃       DataLoader 4        ┃       DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test/bn_forget_ma     │    0.1538461595773697     │                           │                           │
│    test/bn_forget_ppl     │     4544.91748046875      │                           │                           │
│      test/bn_test_ma      │                           │                           │                           │
│     test/bn_test_ppl      │                           │                           │                           │
│     test/hi_forget_ma     │                           │                           │    0.1538461595773697     │
│    test/hi_forget_ppl     │                           │                           │       4231.12890625       │
│      test/hi_test_ma      │                           │    0.15384620428085327    │                           │
│     test/hi_test_ppl      │                           │     4686.43994140625      │                           │
│     test/te_forget_ma     │                           │                           │                           │
│    test/te_forget_ppl     │                           │                           │                           │
│      test/te_test_ma      │                           │                           │                           │
│     test/te_test_ppl      │                           │                           │                           │
└───────────────────────────┴───────────────────────────┴───────────────────────────┴───────────────────────────┘

\n--- Run Complete ---
Metrics saved to: .checkpoints/simplified/flores_metrics.csv
